In [ ]:
from fastai2.basics import *
from fastai2.callback.all import *
from fastai2.vision.all import *

In [ ]:
#all_slow

In [ ]:
#hide
from nbdev.showdoc import *

# Tutorial - Migrating from pure PyTorch

> Incrementally adding fastai goodness to your PyTorch models

### Original PyTorch code

Here's the MNIST training code from the official PyTorch examples (slightly reformatted for space, updated from AdaDelta to AdamW, and converted from a script to a notebook). There's a lot of code!

In [ ]:
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [ ]:
class Net(nn.Sequential):
    def __init__(self):
        super().__init__(
            nn.Conv2d(1, 32, 3, 1), nn.ReLU(),
            nn.Conv2d(32, 64, 3, 1), nn.MaxPool2d(2), nn.Dropout2d(0.25),
            Flatten(), nn.Linear(9216, 128), nn.ReLU(), nn.Dropout2d(0.5),
            nn.Linear(128, 10), nn.LogSoftmax(dim=1) )

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx*len(data), len(train_loader.dataset),
                100. * batch_idx/len(train_loader), loss.item()))

In [ ]:
def test(model, device, test_loader):
    model.eval()
    test_loss,correct = 0,0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss/len(test_loader.dataset), correct, len(test_loader.dataset),
        100. * correct/len(test_loader.dataset)))

In [ ]:
batch_size,test_batch_size = 256,512
epochs,lr,gamma = 1,1e-2,0.7

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = DataLoader(
    datasets.MNIST('../data', train=True, download=True, transform=transforms.Compose([
        transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
        transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=test_batch_size, shuffle=True, **kwargs)

In [ ]:
model = Net().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

## Use the fastai training loop

The most important step is to replace the custom training loop with fastai's. That means you can get rid of `train()`, `test()`, and the epoch loop above, and replace it all with just this:

In [ ]:
data = DataBunch(train_loader, test_loader).cuda()
learn = Learner(data, Net(), loss_func=F.nll_loss, opt_func=Adam, metrics=accuracy)

fastai supports many schedulers. We recommend using 1cycle:

In [ ]:
learn.fit_one_cycle(epochs, lr)

epoch,train_loss,valid_loss,accuracy,time
0,0.134164,0.047528,0.985000,00:12
